In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 라이브러리 설치

In [ ]:
!pip install transformers sentence-transformers langchain openai chromadb bs4 accelerate langchain_community pypdf text_generation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.5 MB/s eta 0:00:

### 데이터 로드

In [ ]:
import zipfile
import os
import pandas as pd

extract_path = '/content/ecommerce/'

# 경로가 없으면 생성
os.makedirs(extract_path, exist_ok=True)
# 압축 해제
with zipfile.ZipFile('/content/drive/MyDrive/llm_recommend/session_group.zip', 'r') as zip_ref:
    zip_ref.extractall(extract_path)


# CSV 파일 경로
csv_file_path = os.path.join(extract_path, 'session_group.csv')

# 로드
df = pd.read_csv(csv_file_path)
# rfm user 데이터 로드
user_rfm = pd.read_csv('/content/drive/MyDrive/llm_recommend/usr_rfm.csv')
# 확인
display(df.head())
display(user_rfm.head())

In [ ]:
import pandas as pd
# amazon review 데이터 로드
amazon = pd.read_csv('/content/drive/MyDrive/llm_recommend/amazon.csv')

# 결측치 처리
amazon.dropna(axis=0, inplace=True)

# 입력오류 행 제거
amazon = amazon[amazon['rating'] != '|']

# float 변환
amazon['rating'] = amazon['rating'].astype(float)

# 통화 기호 및 쉼표 제거 후 float, int 변환
amazon['discounted_price'] = amazon['discounted_price'].replace({'₹': '', ',': ''}, regex=True).astype(float) # 제품 할인 가격(₹)
amazon['actual_price'] = amazon['actual_price'].replace({'₹': '', ',': ''}, regex=True).astype(float)         # 제품 원가(₹)
amazon['discount_percentage'] = amazon['discount_percentage'].replace({'%': ''}, regex=True).astype(int)      # 제품 할인율(%)
amazon['rating_count'] = amazon['rating_count'].replace({',': ''}, regex=True).astype(int)                    # 제품 평가 인원 수

# 카테고리 분리
amazon_df = amazon.copy()
catsplit = amazon_df['category'].str.split('|', expand=True)
# 컬럼명 변경
catsplit = catsplit.rename(columns={0:'category_1', 1:'category_2', 2:'category_3'})

# 복사한 df에 추가

amazon_df['category_1'] = catsplit['category_1']
amazon_df['category_2'] = catsplit['category_2']

amazon_df.drop(columns='category', inplace=True)

# 문자열 값 수정

amazon_df['category_1'] = amazon_df['category_1'].str.replace('&', ' & ')
amazon_df['category_1'] = amazon_df['category_1'].str.replace('OfficeProducts', 'Office Products')
amazon_df['category_1'] = amazon_df['category_1'].str.replace('MusicalInstruments', 'Musical Instruments')
amazon_df['category_1'] = amazon_df['category_1'].str.replace('HomeImprovement', 'Home Improvement')


# 제품 카테고리2 문자열 값 수정

amazon_df['category_2'] = amazon_df['category_2'].str.replace('&', ' & ')
amazon_df['category_2'] = amazon_df['category_2'].str.replace(',', ', ')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('HomeAppliances', 'Home Appliances')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('AirQuality', 'Air Quality')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('WearableTechnology', 'Wearable Technology')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('NetworkingDevices', 'Networking Devices')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('OfficePaperProducts', 'Office Paper Products')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('ExternalDevices', 'External Devices')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('DataStorage', 'Data Storage')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('HomeStorage', 'Home Storage')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('HomeAudio', 'Home Audio')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('GeneralPurposeBatteries', 'General Purpose Batteries')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('BatteryChargers', 'Battery Chargers')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('CraftMaterials', 'Craft Materials')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('OfficeElectronics', 'Office Electronics')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('PowerAccessories', 'Power Accessories')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('CarAccessories', 'Car Accessories')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('HomeMedicalSupplies', 'Home Medical Supplies')
amazon_df['category_2'] = amazon_df['category_2'].str.replace('HomeTheater', 'Home Theater')


# amazon_df 필요한 피처 처리
amazon_product = amazon_df.groupby(['category_1', 'product_id','product_name'], as_index=False)[['discounted_price', 'actual_price']].mean()
amazon_product.columns = ['category_code', 'product_id', 'product_name', 'discounted_price', 'price']
amazon_product.head()

,category_code,product_id,product_name,discounted_price,price
0,Car & Motorbike,B0912WJ87V,Reffair AX30 [MAX] Portable Air Purifier for C...,2339.0,4000.0
1,Computers & Accessories,B002PD61Y4,D-Link DWA-131 300 Mbps Wireless Nano USB Adap...,507.0,1208.0
2,Computers & Accessories,B002SZEOLG,TP-Link Nano USB WiFi Dongle 150Mbps High Gain...,749.0,1339.0
3,Computers & Accessories,B003L62T7W,"Logitech B100 Wired USB Mouse, 3 yr Warranty, ...",279.0,375.0
4,Computers & Accessories,B004IO5BMQ,"Logitech M235 Wireless Mouse, 1000 DPI Optical...",699.0,995.0


In [ ]:
# 제품 구매하지 않은 유저 추출
not_buy = df.loc[df['event_type'].isin(['view', 'cart'])]

# 태깅 및 임베딩위한 데이터 추출
not_buy_user = not_buy.groupby(['session_cluster', 'brand','category_code', 'product_id'], as_index=False)['price'].mean()

not_buy_user['session_cluster'] = not_buy_user['session_cluster'].astype('int')

In [ ]:
# # 제품 구매한 유저 태깅 및 임베딩위한 데이터 추출
buy_user = user_rfm.groupby(['session_cluster', 'custor_segment', 'customer_grade', 'brand','category_code', 'product_id'], as_index=False)['price'].mean()
buy_user['session_cluster'] = buy_user['session_cluster'].astype('int')

### 임베딩 모델

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFace 임베딩 모델
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cuda'},
    encode_kwargs={'normalize_embeddings' : False}
)

<ipython-input-4-69fe98c1077f>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/setti

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Chromdb에 벡터 저장

##### 제품 구매 안한 태깅 및 임베딩 처리

In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

# session_group 태그 처리
not_buy_user['session_group_tag'] = not_buy_user['session_cluster'].astype('category').cat.codes
not_buy_user.drop('session_cluster', axis=1, inplace=True)


# 제품 관련 피처 임베딩
docs = []
for index, row in not_buy_user.iterrows():
    for column in not_buy_user.columns:
        doc = Document(
            page_content=str(row[column]),
            metadata={
                "session_group_tag": row['session_group_tag'],
                "column": column,
                "brand": row['brand'],
                "category_code": row['category_code'],
                "product_id": row['product_id'],
                "price": row['price']
            }
        )
        docs.append(doc)

# 벡터 스토어 생성 및 저장
max_batch_size = 41666
for i in range(0, len(docs), max_batch_size):
    batch_docs = docs[i:i + max_batch_size]
    vectorstore = Chroma.from_documents(
        documents=batch_docs,
        embedding=embeddings,
        persist_directory='./chroma_db',
        collection_name='not_buy_user',
    )

##### 제품 구매한 고객 태깅 및 임베딩 처리

In [ ]:
# 고객 세분화 태그 생성
buy_user['customer_segment_tag'] = (
    buy_user['session_cluster'].astype(str) + '_' +
    buy_user['custor_segment'] + '_' +
    buy_user['customer_grade']
)
# 카테고리 변환
buy_user['customer_segment_tag'] = buy_user['customer_segment_tag'].astype('category').cat.codes

# 세분화에 사용된 열 제거
buy_user.drop(['session_cluster', 'custor_segment', 'customer_grade'], axis=1, inplace=True)


# 제품 관련 피처 임베딩
docs2 = []
for index, row in buy_user.iterrows():
    for column in buy_user.columns:
        doc2 = Document(
            page_content=str(row[column]),
            metadata={
                "customer_segment_tag": row['customer_segment_tag'],
                "column": column,
                "brand": row['brand'],
                "category_code": row['category_code'],
                "product_id": row['product_id'],
                "price": row['price']
            }
        )
        docs2.append(doc2)

# 벡터 스토어 생성 및 저장
max_batch_size = 41666
for i in range(0, len(docs2), max_batch_size):
    batch_docs = docs2[i:i + max_batch_size]
    vectorstore2 = Chroma.from_documents(
        documents=batch_docs,
        embedding=embeddings,
        persist_directory='./chroma_db',
        collection_name='buy_user',
    )

##### amazon_df 제품 정보 임베딩 처리

In [ ]:
# 제품 관련 피처 임베딩
docs3 = []
for index, row in amazon_product.iterrows():
    for column in amazon_product.columns:
        doc3 = Document(
            page_content=str(row[column]),
        metadata={
            "category_code": row['category_code'],
            "product_id": row['product_id'],
            "discounted_price": row['discounted_price'],
            "price": row['price']
        }
    )
    docs3.append(doc3)

# 벡터 스토어 생성 및 저장
vectorstore3 = Chroma.from_documents(
    documents=docs3,
    embedding=embeddings,
    persist_directory='./amazon_chroma_db',
    collection_name='amazon_product',
)

In [ ]:
import shutil
import os

# 압축할 원본 폴더 경로
source_folder = '/content/chroma_db'

# 압축 파일 경로
zip_file_path = '/content/chroma_db.zip'

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', source_folder)

'/content/chroma_db.zip'

In [ ]:
# 목적지 폴더 경로
destination = '/content/drive/MyDrive/llm_recommend/chroma_db.zip'

# 압축된 파일 이동
shutil.move(zip_file_path, destination)

'/content/drive/MyDrive/llm_recommend/chroma_db.zip'

In [ ]:
import shutil
import os

# 압축할 원본 폴더 경로
source_folder2 = '/content/amazon_chroma_db'

# 압축 파일 경로
zip_file_path2 = '/content/amazon_chroma_db.zip'

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_path2.replace('.zip', ''), 'zip', source_folder2)

# 목적지 폴더 경로
destination2 = '/content/drive/MyDrive/llm_recommend/amazon_chroma_db.zip'

# 압축된 파일 이동
shutil.move(zip_file_path2, destination2)

'/content/drive/MyDrive/llm_recommend/amazon_chroma_db.zip'

### Chromadb zip 파일 압축 해제 및 로드

In [ ]:
import zipfile
import os
from langchain.vectorstores import Chroma

# 압축 파일 경로
zip_file_path = '/content/drive/MyDrive/llm_recommend/chroma_db.zip'
# 압축을 풀 디렉토리
extract_dir = '/content/chroma_db/'

# 디렉토리 생성 (존재하지 않는 경우)
os.makedirs(extract_dir, exist_ok=True)

# 압축 파일 열기 및 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


# 디스크에서 문서 로드
vectorstore = Chroma(
    persist_directory='/content/chroma_db',
    embedding_function=embeddings,
    collection_name="not_buy_user",
)

# 디스크에서 문서 로드
vectorstore2 = Chroma(
    persist_directory='/content/chroma_db',
    embedding_function=embeddings,
    collection_name="buy_user",
)

<ipython-input-5-e5facd9dd5d1>:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [ ]:
# 압축 파일 경로
zip_file_path2 = '/content/drive/MyDrive/llm_recommend/amazon_chroma_db.zip'
# 압축을 풀 디렉토리
extract_dir2 = '/content/amazon_chroma_db/'

# 디렉토리 생성 (존재하지 않는 경우)
os.makedirs(extract_dir2, exist_ok=True)

# 압축 파일 열기 및 해제
with zipfile.ZipFile(zip_file_path2, 'r') as zip_ref:
    zip_ref.extractall(extract_dir2)

# 디스크에서 문서 로드
amazon_vectorstore = Chroma(
    persist_directory='/content/amazon_chroma_db',
    embedding_function=embeddings,
    collection_name="amazon_product",
)

### Retriever 생성
#### MMR(Maximal Marginal Relevance) 방법 사용 - 다양성 중시

In [ ]:
not_buy_retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':3})
buy_retriever = vectorstore2.as_retriever(search_type='mmr', search_kwargs={'k':3})
amazon_retriever = amazon_vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':3})

In [ ]:
from langchain.retrievers import EnsembleRetriever
# 이커머스 제품을 구매하지 않은, 했던 유저 벡터스토어 앙상블
reference_retriever = EnsembleRetriever(
    retrievers=[not_buy_retriever, buy_retriever],
    weights=[0.4, 0.6]
)

### LLM 로드
- HugginfFace

In [ ]:
!pip install langchain-huggingface

In [ ]:
import os
from langchain_huggingface import HuggingFaceEndpoint

# 환경 변수에 토큰 설정
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "본인의 key"

# 환경 변수에서 Hugging Face API 토큰 로딩
HF_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm = HuggingFaceEndpoint(
    endpoint_url='https://api-inference.huggingface.co/models/google/gemma-7b',  # endpoint_url 사용
    model_kwargs={
        'model_id': 'google/gemma-7b'  # model_id를 model_kwargs로 전달
    },
    max_new_tokens=1024,
    top_k=50,
    temperature=0.1,
    repetition_penalty=1.03,
    server_kwargs={
        "headers": {
            "Authorization": f"Bearer {HF_TOKEN}",
            "Content-Type": "application/json",
        }
    },
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Prompt 템플릿 생성 및 QA 체인 생성

### amazon retriever만 사용해 추천

QnA챗봇

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, MessagesPlaceholder


# Prompt 생성
prompt = PromptTemplate.from_template(
    """You are a chatbot that recommends certain products to users based on the context provided.
    First of all, you can have a natural conversation.
    Consider factors such as product_id, price, brand, and category_code.
    Recommend the top three products and provide a brief explanation for each recommendation.
    Make sure you give at least three recommendations and if not possible state the reason why and suggest the maximum number of recommendations available.

    If you cannot provide a recommendation based on the given information,
    respond with: 'I am sorry, I don't have enough information to make a recommendation at this time.'
#Question:
{question}
#Context:
{context}

#Answer:
"""
)

# RAG Chain 연결
rag_chain = (
    {'context': amazon_retriever, 'question': RunnablePassthrough()}
    | prompt # prompt : LLM 설정
    | llm  # model : LLM 종류
    | StrOutputParser() # 채팅을 문자열로 변환하는 간단한 출력 구문 분석기
)

In [ ]:
# 질문 입력
question = input("상품 추천받고 싶은거에 대해 입력하세요.")

# RAG 체인 실행
response = rag_chain.invoke(question)
print(response)

상품 추천받고 싶은거에 대해 입력하세요.hi. do you recommend clothes?
Hi! I would be happy help you find some clothes in our store. Here is my recommendation list:


Recommendation #1 - The first item I would recommend is [Product Name] (link). This item has a discounted price of $XX which is a great deal considering its original price was $XXX. It also comes from a reputable brand called [Brand]. This particular model is part of the Office Products category code so it should fit your needs perfectly.


In conclusion, I hope these recommendations were helpful but please let me know what else we can do for you today by replying back here or contacting us directly through our website/phone line etc.. Thank you again :)


invoke시 page_contet 사용해서 성능 평가

In [ ]:
# 임베딩 함수 정의
def embedding_function(doc):
    # Document 객체에서 텍스트를 추출하여 임베딩을 생성
    return embeddings.embed_documents([doc.page_content])[0]  # page_content 속성을 사용

# 질문 입력
question = input("상품 추천받고 싶은거에 대해 입력하세요.")

# 추천 결과 수집
predicted_results = amazon_retriever.invoke(question)
reference_results = not_buy_retriever.invoke(question) + buy_retriever.invoke(question)

# MRR 계산
mrr = 0
for rank, result in enumerate(predicted_results, start=1):
    if result in reference_results:
        mrr = 1 / rank
        break

# 코사인 유사도 계산 (임베딩 벡터 비교)
predicted_embeddings = [embedding_function(doc) for doc in predicted_results]
reference_embeddings = [embedding_function(doc) for doc in reference_results]
cosine_similarities = cosine_similarity(predicted_embeddings, reference_embeddings)

# 결과 출력
print(f"MRR: {mrr}")
print(f"Average Cosine Similarity: {cosine_similarities.mean()}")

상품 추천받고 싶은거에 대해 입력하세요.iphone
MRR: 0
Average Cosine Similarity: 0.4855216050921575


invoke시 metadata중 일치하는 컬럼명만 사용해서 성능 평가

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

# 임베딩 함수 정의 (metadata 사용)
def embedding_function(doc):
    # Document 객체에서 metadata를 문자열로 변환하여 임베딩을 생성
    metadata_str = f"{doc.metadata['product_id']} {doc.metadata.get('category_code', '')} {doc.metadata.get('price', '')}"
    return embeddings.embed_documents([metadata_str])[0]

# 질문 입력
question = input("상품 추천받고 싶은거에 대해 입력하세요.")

# 추천 결과 수집
predicted_results = amazon_retriever.invoke(question)
reference_results = not_buy_retriever.invoke(question) + buy_retriever.invoke(question)

# MRR 계산
mrr = 0
predicted_metadata = [doc.metadata for doc in predicted_results]
reference_metadata = [doc.metadata for doc in reference_results]

# MRR 계산
for rank, result in enumerate(predicted_metadata, start=1):
    if result in reference_metadata:
        mrr = 1 / rank
        break

# 코사인 유사도 계산 (임베딩 벡터 비교)
predicted_embeddings = [embedding_function(doc) for doc in predicted_results]
reference_embeddings = [embedding_function(doc) for doc in reference_results]
cosine_similarities = cosine_similarity(predicted_embeddings, reference_embeddings)

# 결과 출력
print(f"MRR: {mrr}")
print(f"Average Cosine Similarity: {cosine_similarities.mean()}")


상품 추천받고 싶은거에 대해 입력하세요.iphone
MRR: 0
Average Cosine Similarity: 0.5353824214211796


invoke시 metadata 사용해서 성능 평가

In [ ]:
# 임베딩 함수 정의 (전체 metadata 사용)
def embedding_function(doc):
    # Document 객체에서 모든 metadata를 문자열로 변환하여 임베딩을 생성
    metadata_str = ' '.join(f"{key}: {value}" for key, value in doc.metadata.items())
    return embeddings.embed_documents([metadata_str])[0]

# 질문 입력
question = input("상품 추천받고 싶은거에 대해 입력하세요.")

# 추천 결과 수집
predicted_results = amazon_retriever.invoke(question)
reference_results = not_buy_retriever.invoke(question) + buy_retriever.invoke(question)

# MRR 계산
mrr = 0
predicted_metadata = [doc.metadata for doc in predicted_results]
reference_metadata = [doc.metadata for doc in reference_results]

# metadata의 product_id를 사용하여 MRR 계산
predicted_ids = {doc.metadata['product_id'] for doc in predicted_results}
reference_ids = {doc.metadata['product_id'] for doc in reference_results}

for rank, result in enumerate(predicted_results, start=1):
    if result.metadata['product_id'] in reference_ids:
        mrr = 1 / rank
        break

# 코사인 유사도 계산 (임베딩 벡터 비교)
predicted_embeddings = [embedding_function(doc) for doc in predicted_results]
reference_embeddings = [embedding_function(doc) for doc in reference_results]
cosine_similarities = cosine_similarity(predicted_embeddings, reference_embeddings)

# 결과 출력
print(f"MRR: {mrr}")
print(f"Average Cosine Similarity: {cosine_similarities.mean()}")


상품 추천받고 싶은거에 대해 입력하세요.iphone
MRR: 0
Average Cosine Similarity: 0.7079970201380038


생성 문장 기반 성능 검증

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Prompt 생성
prompt = PromptTemplate.from_template(
    """You are a chatbot that recommends certain products to users based on the context provided.
    First of all, you can have a natural conversation.
    Consider factors such as product_id, price, brand, and category_code.
    Recommend the top three products and provide a brief explanation for each recommendation.
    Make sure you give at least three recommendations and if not possible state the reason why and suggest the maximum number of recommendations available.

    If you cannot provide a recommendation based on the given information,
    respond with: 'I am sorry, I don't have enough information to make a recommendation at this time.'
#Question:
{question}
#Context:
{context}

#Answer:
"""
)

# RAG Chain 설정
rag_chain_amazon = (
    {'context': amazon_retriever, 'question': RunnablePassthrough()}
    | prompt  # Prompt 설정
    | llm  # LLM 종류
    | StrOutputParser()  # 출력 구문 분석기
)

rag_chain_reference = (
    {'context': reference_retriever, 'question': RunnablePassthrough()}
    | prompt  # Prompt 설정
    | llm  # LLM 종류
    | StrOutputParser()  # 출력 구문 분석기
)

# 질문 입력
question = input("상품 추천받고 싶은거에 대해 입력하세요.")

# RAG Chain을 통해 예측 결과 수집
predicted_results = rag_chain_amazon.invoke(question)
reference_results = rag_chain_reference.invoke(question)

# 문장으로 분리
predicted_sentences = predicted_results.split('. ')
reference_sentences = reference_results.split('. ')

# 수정된 embedding_function
def embedding_function(doc):
    # 문자열을 직접 사용하여 임베딩을 생성
    return embeddings.embed_documents([doc])[0]

# MRR 및 코사인 유사도 계산
mrr_total = 0  # 전체 MRR을 누적할 변수
total_cosine_similarity = 0
num_queries = len(predicted_sentences)  # 쿼리 수

for predicted_sentence in predicted_sentences:
    predicted_embedding = embedding_function(predicted_sentence)
    max_similarity = float('-inf')  # 최대 유사도를 위한 초기값
    relevant_rank = None  # MRR 계산을 위한 랭크 변수

    for rank, reference_sentence in enumerate(reference_sentences, start=1):
        reference_embedding = embedding_function(reference_sentence)

        # 코사인 유사도 계산
        similarity = cosine_similarity([predicted_embedding], [reference_embedding])[0][0]

        # 최대 유사도 업데이트
        if similarity > max_similarity:
            max_similarity = similarity
            relevant_rank = rank  # 현재 랭크 저장

    # MRR 계산
    if relevant_rank is not None:
        mrr_total += 1 / relevant_rank  # 유효한 rank가 존재할 경우만 업데이트

    total_cosine_similarity += max_similarity  # 최대 유사도를 총합에 추가

# 평균 MRR 계산
mrr_average = mrr_total / num_queries if num_queries > 0 else 0

# 평균 코사인 유사도 계산
average_cosine_similarity = total_cosine_similarity / num_queries if num_queries > 0 else 0

# 결과 출력
print(f"MRR: {mrr_average}")
print(f"Average Cosine Similarity: {average_cosine_similarity}")

상품 추천받고 싶은거에 대해 입력하세요.Do you recommend laptop?
MRR: 1.0
Average Cosine Similarity: 0.6115924575834322


1. 결과 수집:
rag_chain_amazon과 rag_chain_reference를 통해 예측 및 참조 결과를 수집.
2. 문장 분리:
예측 결과와 참조 결과를 문장 단위로 나눔.
3. 임베딩 생성:
각 문장을 임베딩하여 벡터로 변환.
4. MRR 및 코사인 유사도 계산:
각 예측 문장에 대해 참조 문장과의 유사도를 계산하고, 관련성이 높은 문장의 순위를 반영하여 MRR을 업데이트.
최대 유사도를 누적하여 평균 코사인 유사도를 계산.

Mean Reciprocal Rank (MRR) : 사용자의 쿼리에 대한 가장 관련성 높은 결과의 순위의 역수를 평균한 값.

계산 방법:

각 쿼리에서 가장 관련성이 높은 결과의 순위를 찾고, 그 순위의 역수를 구함.
여러 쿼리에 대해 평균을 계산.

의미:

MRR이 1에 가까울수록 시스템이 관련성 높은 결과를 잘 제공함을 의미.

평균 코사인 유사도 (Average Cosine Similarity) : 예측된 문장과 참조 문장 간의 유사성을 측정하는 지표.

계산 방법:

각 예측 문장에 대해 모든 참조 문장과의 코사인 유사도를 계산.
각 예측 문장에 대해 최대 유사도를 찾아 총합.
쿼리 수로 나누어 평균을 구함.

의미:

평균 코사인 유사도가 1에 가까울수록 예측과 참조 문장이 유사함을 나타냄.

In [ ]:
print("Predicted Results:", predicted_results)
print("Reference Results:", reference_results)

Predicted Results: Hi! I would recommend the following laptops from our selection: [document('https://www.amazon.com/dp/B09F...')](https://docs.google.com/.../viewform),  [laptop] (https://docs.......) , [Laptop](http...). The first one is great for gaming or video editing due it has a powerful processor and graphics card. The second option offers good value for money because it comes equipped wtih an Intel Core i5 CPU which makes it suitable for everyday tasks like browsing websites online etc., while still being affordable compared other models out there today . Finally third choice provides excellent performance thanks its AMD Ryzen™ APU technology combined together perfectly well balanced set up between speed power consumption efficiency making them ideal solution when looking after both work life balance needs
Reference Results: Here is my recommendation:
- The first one is pocketbook which is in computers ebooks category code and has a price of $1\n$54.\nThis item is recommended 